In [ ]:
import torch

In [ ]:
# Datos (tensores PyTorch)
# ----------------------------
x = torch.tensor([50.0, 80.0, 120.0])
y = torch.tensor([150.0, 200.0, 280.0])

In [ ]:
# (1) GRADIENT DESCENT "A MANO" (sin librerías de optimización)
# ------------------------------------------------------------
def manual_gradient_descent(x, y, lr=0.0001, epochs=10):
    # Inicialización
    w = torch.tensor(0.0)
    b = torch.tensor(0.0)
    n = x.numel()
    idx = torch.arange(n)
    global_iter = 0
    print("=== (1) Manual GD")
    print(f"lr={lr}, epochs={epochs}\n")

    for epoch in range(1, epochs + 1):
        for j in idx:
          global_iter += 1
          xi = x[j]
          yi = y[j]

          y_hat = w * xi + b
          e = y_hat - yi

          # Loss por muestra (opcional)
          loss_i = 0.5 * (e ** 2)

          # Gradientes por muestra:
          # L = 1/2 (e^2)  => dL/dw = e*xi, dL/db = e
          dw = e * xi
          db = e

          # Actualización SGD
          w = w - lr * dw
          b = b - lr * db

          print(f"Iter {global_iter:02d} | epoch={epoch:02d} | x={xi.item():.0f} y={yi.item():.0f} "
                  f"| loss={loss_i.item():.6f} | w={w.item():.9f} | b={b.item():.9f}")

        print(f"--> End epoch {epoch:02d}: w={w.item():.9f}, b={b.item():.9f}\n")
    return w, b

In [ ]:
w1, b1 = manual_gradient_descent(x, y, lr=0.0001, epochs=100)

=== (1) Manual GD
lr=0.0001, epochs=100

Iter 01 | epoch=01 | x=50 y=150 | loss=11250.000000 | w=0.750000000 | b=0.015000000
Iter 02 | epoch=01 | x=80 y=200 | loss=9797.900391 | w=1.869879961 | b=0.028998500
Iter 03 | epoch=01 | x=120 y=280 | loss=1544.869385 | w=2.536904812 | b=0.034557041
--> End epoch 01: w=2.536904812, b=0.034557041

Iter 04 | epoch=02 | x=50 y=150 | loss=267.272034 | w=2.652505875 | b=0.036869060
Iter 05 | epoch=02 | x=80 y=200 | loss=74.876190 | w=2.554607153 | b=0.035645328
Iter 06 | epoch=02 | x=120 y=280 | loss=353.474182 | w=2.235545158 | b=0.032986477
--> End epoch 02: w=2.235545158, b=0.032986477

Iter 07 | epoch=03 | x=50 y=150 | loss=729.228516 | w=2.426493883 | b=0.036805451
Iter 08 | epoch=03 | x=80 y=200 | loss=17.074350 | w=2.473243475 | b=0.037389819
Iter 09 | epoch=03 | x=120 y=280 | loss=141.567215 | w=2.271324396 | b=0.035707157
--> End epoch 03: w=2.271324396, b=0.035707157

Iter 10 | epoch=04 | x=50 y=150 | loss=662.409790 | w=2.453314781 | b=0.

In [ ]:
def torch_sgd(x, y, lr=1e-4, epochs=10):
    # Parámetros como tensores entrenables
    w = torch.zeros((), requires_grad=True)  # escalar
    b = torch.zeros((), requires_grad=True)  # escalar

    optimizer = torch.optim.SGD([w, b], lr=lr)

    print("\n=== (2) PyTorch autograd + optim.SGD ===")
    print(f"lr={lr}, epochs={epochs}\n")

    for epoch in range(1, epochs + 1):
        optimizer.zero_grad()

        y_hat = w * x + b
        loss = torch.mean((y_hat - y) ** 2)  # MSE

        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch:02d} | loss={loss.item():.6f} | w={w.item():.9f} | b={b.item():.9f}")

    return w, b

In [ ]:
w2, b2 = torch_sgd(x, y, lr=1e-4, epochs=100)


=== (2) PyTorch autograd + optim.SGD ===
lr=0.0001, epochs=100

Epoch 01 | loss=46966.667969 | w=3.806666613 | b=0.041999999
Epoch 02 | loss=14613.083008 | w=1.699611187 | b=0.020547157
Epoch 03 | loss=4700.724609 | w=2.865872622 | b=0.034216195
Epoch 04 | loss=1663.810059 | w=2.220313549 | b=0.028444810
Epoch 05 | loss=733.364197 | w=2.577619076 | b=0.033433896
Epoch 06 | loss=448.289398 | w=2.379826546 | b=0.032466888
Epoch 07 | loss=360.939911 | w=2.489288092 | b=0.034796618
Epoch 08 | loss=334.168427 | w=2.428680658 | b=0.035301525
Epoch 09 | loss=325.957245 | w=2.462208271 | b=0.036816452
Epoch 10 | loss=323.432098 | w=2.443631172 | b=0.037772283
Epoch 11 | loss=322.649567 | w=2.453894377 | b=0.039037541
Epoch 12 | loss=322.400116 | w=2.448194504 | b=0.040131494
Epoch 13 | loss=322.314514 | w=2.451330423 | b=0.041320227
Epoch 14 | loss=322.279144 | w=2.449575186 | b=0.042456456
Epoch 15 | loss=322.259155 | w=2.450527430 | b=0.043621711
Epoch 16 | loss=322.243500 | w=2.449981213 |

In [ ]:
# y = w x + b  =>  [x  1] [w b]^T ≈ y
A = torch.stack([x, torch.ones_like(x)], dim=1)          # (N,2)
sol = torch.linalg.lstsq(A, y).solution                  # (2,)
w_opt, b_opt = sol[0].item(), sol[1].item()

print("\n=== Verificación (óptimo por mínimos cuadrados) ===")
print(f"w_opt = {w_opt:.9f}, b_opt = {b_opt:.9f}")


=== Verificación (óptimo por mínimos cuadrados) ===
w_opt = 1.864864349, b_opt = 54.594699860
